In [239]:
import sagemaker
import boto3
import json
import subprocess
from langchain_aws import ChatBedrockConverse
from os import environ
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import OpenSearchVectorSearch
from opensearchpy import RequestsHttpConnection, AWSV4SignerAuth
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel, ConfigurableField

import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from typing import TypedDict
from operator import itemgetter

In [240]:
# Login AWS
aws_profile = "vrt-analytics-engineer-nonsensitive"
envvars = subprocess.check_output(['aws-vault', 'exec', aws_profile, '--', 'env'])
for envline in envvars.split(b'\n'):
    line = envline.decode('utf8')
    eqpos = line.find('=')
    if eqpos < 4:
        continue
    k = line[0:eqpos]
    v = line[eqpos+1:]
    if k == 'AWS_ACCESS_KEY_ID':
        aws_access_key_id = v
    if k == 'AWS_SECRET_ACCESS_KEY':
        aws_secret_access_key = v
    if k == 'AWS_SESSION_TOKEN':
        aws_session_token = v

session = boto3.Session(
aws_access_key_id, aws_secret_access_key, aws_session_token,region_name="eu-west-1"
)
credentials = session.get_credentials()
sagemaker_session = sagemaker.Session(boto_session=session)
role = sagemaker.get_execution_role(sagemaker_session=sagemaker_session)
auth = AWSV4SignerAuth(credentials, 'eu-west-2', 'aoss')


environ["AWS_ACCESS_KEY_ID"] = aws_access_key_id
environ["AWS_SECRET_ACCESS_KEY"] = aws_secret_access_key
environ["AWS_SESSION_TOKEN"] = aws_session_token

In [241]:
# LLM endpoint
llm = ChatBedrockConverse(
    region_name="eu-west-2",
    model="meta.llama3-70b-instruct-v1:0",
    temperature=0.6,
    top_p=0.6,
    max_tokens=512
)

model_name = "NetherlandsForensicInstitute/robbert-2022-dutch-sentence-transformers"
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    encode_kwargs=encode_kwargs
)

# Init OpenSearch client connection
docsearch = OpenSearchVectorSearch(
    index_name="vrtmax-catalog-index",  # TODO: use the same index-name used in the ingestion script
    embedding_function=embeddings,
    opensearch_url="https://epcavlvwitam2ivpwv4k.eu-west-2.aoss.amazonaws.com:443",  # TODO: e.g. use the AWS OpenSearch domain instantiated previously
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    is_appx_search=False
)
# retriever = docsearch.as_retriever(search_kwargs={'k': 5,"vector_field":"vrtmax_catalog_vector"})

In [270]:
user_input = "terzake"

In [271]:
# Fuzzy matching
programs = pd.read_csv("data/programs.csv")


# Initialize filter dict
boolean_filter = {"bool":{"must":[]}}

programs_list = list(programs.naam)
output = []
for program in programs_list:
    try:
        score = fuzz.partial_ratio(program.lower(),user_input.lower())
    except:
        score = 0
    output.append((program,score))


output = sorted(
    output, 
    key=lambda x: x[1],
    reverse=True
)[0]

if output[1] > 95:
    program_match = output[0]
    boolean_filter["bool"]["must"].append({"match_phrase" : {"metadata.mediacontent_pagetitle_program":program_match}})


In [272]:
boolean_filter

{'bool': {'must': [{'match_phrase': {'metadata.mediacontent_pagetitle_program': 'Terzake'}}]}}

In [273]:
# Docsearch (ideally in langchain, can't get it to work)
docs = docsearch.similarity_search(
    user_input,
    k=5,
    vector_field="vrtmax_catalog_vector",
    search_type="script_scoring",
    pre_filter=boolean_filter
)

In [274]:
# Create context
def format_docs(docs):
    context = []
    for d in docs:
        program_description = ""
        if d.metadata["mediacontent_page_description_program"]  != "":
            program_description = docs[0].metadata["mediacontent_page_description_program"]
        elif d.metadata["mediacontent_page_editorialtitle_program"]  != "":
            program_description = docs[0].metadata["mediacontent_page_editorialtitle_program"]

        context.append("Het programma met de naam " + d.metadata["mediacontent_pagetitle_program"] +\
                " heeft volgende beschrijving: " + program_description  +\
                " De episode van dit programma heeft als beschrijving: " + d.metadata["mediacontent_page_description"])
        
    return "\n\n".join(context)

context = format_docs(docs)

In [275]:
# Create system and user prompt

system_prompt = """
Je bent een hulpvaardige assistent die Nederlands spreekt.
Je krijgt vragen over de catalogus van programma's van het videoplatform VRT MAX. 
Je probeert mensen te helpen om programma's aan te bevelen die aansluiten bij hun vraag.
Daarvoor krijg je een beschrijving van een aantal programma's.
Aan jou om wat relevant is aan te bevelen.
Begin niet met het geven van jouw mening over de programma's. Begin direct met het aanbevelen van relevante content aan de gebruiker.
"""

message = """Beantwoordt de vraag op basis van de volgende context:

""" + context + """

Vraag: {question}
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", message),
    ]
)

chain = (
    {"question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke(user_input)

'\n\nU zoekt naar programma\'s over actuele thema\'s en nieuws uit binnen- en buitenland. Ik beveel het programma "Terzake" aan, dat dagelijks actuele thema\'s en nieuws behandelt. Het programma biedt een breed scala aan onderwerpen, van politiek en economie tot cultuur en milieu. U kunt erop rekenen dat u op de hoogte blijft van de laatste ontwikkelingen in binnen- en buitenland.'

In [119]:
boolean_filter

{'bool': {'must': [{'match': {'metadata.mediacontent_pagetitle_program': 'De afspraak'}}]}}

In [118]:
docs = docsearch.similarity_search(
        "",
        5,
        vector_field="vrtmax_catalog_vector",
        boolean_filter=boolean_filter
    )    
docs

[Document(metadata={'mediacontent_page_description_program': '', 'mediacontent_page_description': 'Captaties van een brede waaier van artiesten in de legendarische Studio Toots. Niet enkel muziek, maar ook podiumkunsten, comedy, theater, ... om de cultuursector te ondersteunen. Vandaag: Sarah Vanhee en Camila Lakhloufi.', 'mediacontent_page_editorialtitle_program': 'Vlaamse artiesten treden op in de Toots-studio', 'mediacontent_pagetitle_program': 'De Toots Sessies', 'mediacontent_pagetitle_season': 'Seizoen 1', 'mediacontent_pagetitle': 'Sarah Vanhee en Camila Lakhloufi', 'offering_publication_planneduntil': '2025-01-22 22:59:00.000', 'brand_contentbrand': 'canvas', 'mediacontent_pageurl': 'https://www.vrt.be/vrtmax/a-z/de-toots-sessies/2020/de-toots-sessies-s2020a2/', 'mediacontent_imageurl': '//images.vrt.be/orig/2020/11/17/61488107-28c2-11eb-aae0-02b7b76bf47f.jpg', 'mediacontent_programimageurl': '//images.vrt.be/orig/2022/11/18/dfe0327e-6775-11ed-b07d-02b7b76bf47f.jpg', 'mediacont

In [109]:
boolean_filter = {"bool":{"must":[
    {"match_phrase" : {"metadata.mediacontent_page_description_program":"Peter Wijninga"}},
    {"match" : {"metadata.brand_contentbrand":"vrtnws"}},
]}}

docsearch.similarity_search(
    user_input,
    5,
    vector_field="vrtmax_catalog_vector",
    boolean_filter=boolean_filter
)

[Document(metadata={'mediacontent_page_description_program': 'Aanval Iran op Israël170 drones, 30 kruisraketten en 120 ballistische raketten stuurde Iran naar Israël, volgens het Israëlische leger. Door verschillende afweersystemen zoals de ‘Iron Dome’ en ‘David’s Sling’ werd de aanval afgeweerd. Wat gaat Israël nu doen? Willem Van Mullem interviewt militair expert Peter Wijninga (HCSS). We gaan naar correspondent Nasrah Habiballah in Tel Aviv en historicus Peyman Jafari in Washtington. Rudi Vranckx geeft meer duiding in de studio.Het conflict tussen Iran en Israël kent een lange geschiedenis. Een overzicht door Bram Vandeputte.Notre Dame herrijstDe wereldberoemde Notre Dame in Parijs werd in 2019 getroffen door een heel zware brand. President Macron wou er alles aan doen om de kathedraal zo snel mogelijk te restaureren, binnen de vijf jaar. En dat lijkt te lukken. Steven Decraene trok naar Parijs.', 'mediacontent_page_description': 'Politieke formatieDe herwerkte versie van de ‘supern

In [90]:
# Get retriever
def get_retriever():
    retriever = docsearch.as_retriever(
        k="5",
        search_type="similarity",
        vector_field ="vrtmax_catalog_vector",
    )
    configurable_retriever = retriever.configurable_fields(
        search_kwargs=ConfigurableField(
            id="search_kwargs",
            name="Search Kwargs",
            description="The search kwargs to use. Includes dynamic category adjustment.",
        )
    )
    return configurable_retriever

In [97]:
class SemanticQueryInput(TypedDict):
    question: str


def get_chain():
    retriever_chain = (
        RunnableParallel(
            context=(
                itemgetter("question")
                | get_retriever()
            ),
            question=itemgetter("question"),
        )
        | RunnableParallel(docs=itemgetter("context"), question=itemgetter("question"))
        # | enrich_docs_chain
    ).with_types(input_type=SemanticQueryInput)
    return retriever_chain

chain = get_chain()
chain.invoke({"question":  user_input}, config={"configurable": {"search_kwargs":   {"k":1,"vector_field":"vrtmax_catalog_vector","filter": {"bool": {"must": [{"match": {"brand_contentbrand": "vrtnws"}}]}}}}})

{'docs': [], 'question': 'aflevering van De afspraak'}

In [60]:
chain

RunnableBinding(bound={
  context: RunnableLambda(itemgetter('question'))
           | RunnableConfigurableFields(default=VectorStoreRetriever(tags=['OpenSearchVectorSearch', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.opensearch_vector_search.OpenSearchVectorSearch object at 0x7ea1f95bf760>), fields={'search_kwargs': ConfigurableField(id='search_kwargs', name='Search Kwargs', description='The search kwargs to use. Includes dynamic category adjustment.', annotation=None, is_shared=False)}),
  question: RunnableLambda(itemgetter('question'))
}
| {
    docs: RunnableLambda(itemgetter('context')),
    question: RunnableLambda(itemgetter('question'))
  }, custom_input_type=<class '__main__.SemanticQueryInput'>)

In [18]:
docsearch.similarity_search("iets over geld en beleggen", 5,vector_field="vrtmax_catalog_vector")

[Document(metadata={'mediacontent_page_description_program': '', 'mediacontent_page_description': 'Wat is een blockchain? Hét woord van de voorbije jaren: blockchain! En na deze video met wiskundige An Braeken (VUB) snap jij ook perfect wat de blockchain is.', 'mediacontent_page_editorialtitle_program': 'Krijg simpele antwoorden op wetenschappelijke vragen', 'mediacontent_pagetitle_program': 'WetenSNAP', 'mediacontent_pagetitle_season': '2022-2023', 'mediacontent_pagetitle': 'Wat is een blockchain?', 'offering_publication_planneduntil': '2032-12-31 22:59:00.000', 'brand_contentbrand': 'radio1', 'mediacontent_pageurl': 'https://www.vrt.be/vrtnu/a-z/wetensnap/2022-2023/wetensnap-s2022-2023a19/', 'mediacontent_imageurl': '//images.vrt.be/orig/2022/10/26/84c3f8c6-5525-11ed-b07d-02b7b76bf47f.png', 'mediacontent_programimageurl': '//images.vrt.be/orig/2022/09/29/bae33113-400c-11ed-b07d-02b7b76bf47f.jpg', 'mediacontent_episode_castlist': '[]'}, page_content="Iets met je bankkaart kopen, geld 

In [32]:
query = "iets over geld en beleggen"
# filterterm =  {"bool":{
#     "filter": {
#   "term": {
#     "metadata.brand_contentbrand.keyword": "vrtnws"
#   }
#   }
# }
# }

# https://opensearch.org/docs/latest/query-dsl/compound/bool/
boolean_filter = {"bool":{"must":[
    {"match_phrase" : {"metadata.mediacontent_page_description_program":"Peter Wijninga"}},
    {"match" : {"metadata.brand_contentbrand":"vrtnws"}},
]}}

docsearch.similarity_search(
    query,
    5,
    vector_field="vrtmax_catalog_vector",
    boolean_filter=boolean_filter
)

[Document(metadata={'mediacontent_page_description_program': 'Aanval Iran op Israël170 drones, 30 kruisraketten en 120 ballistische raketten stuurde Iran naar Israël, volgens het Israëlische leger. Door verschillende afweersystemen zoals de ‘Iron Dome’ en ‘David’s Sling’ werd de aanval afgeweerd. Wat gaat Israël nu doen? Willem Van Mullem interviewt militair expert Peter Wijninga (HCSS). We gaan naar correspondent Nasrah Habiballah in Tel Aviv en historicus Peyman Jafari in Washtington. Rudi Vranckx geeft meer duiding in de studio.Het conflict tussen Iran en Israël kent een lange geschiedenis. Een overzicht door Bram Vandeputte.Notre Dame herrijstDe wereldberoemde Notre Dame in Parijs werd in 2019 getroffen door een heel zware brand. President Macron wou er alles aan doen om de kathedraal zo snel mogelijk te restaureren, binnen de vijf jaar. En dat lijkt te lukken. Steven Decraene trok naar Parijs.', 'mediacontent_page_description': 'Strijd om geld van de staatsbonDe strijd om de 22 mi

In [68]:
retriever = docsearch.as_retriever()

In [25]:
def format_docs(docs):
    context = []
    for d in docs:
        program_description = ""
        if d.metadata["mediacontent_page_description_program"]  != "":
            program_description = docs[0].metadata["mediacontent_page_description_program"]
        elif d.metadata["mediacontent_page_editorialtitle_program"]  != "":
            program_description = docs[0].metadata["mediacontent_page_editorialtitle_program"]

        context.append("Het programma met de naam " + d.metadata["mediacontent_pagetitle_program"] +\
                " heeft volgende beschrijving: " + program_description  +\
                " De episode van dit programma heeft als beschrijving: " + d.metadata["mediacontent_page_description"])
        
    return "\n\n".join(context)

'\n\nOp basis van uw vraag zou ik het programma "Boekenmarathon en Boekenfeest 2020" aanbevelen, omdat het programma specifiek gaat over geldbeleggen en hoe je goede investeringskeuzes kunt maken. Ook het programma "Schermtijd" zou een goede optie kunnen zijn, omdat het programma gaat over veilig beleggen met een app. Als u meer algemene informatie zoekt over de economische actualiteit, zou "De markt" ook een goede keuze kunnen zijn.'

: 

In [4]:
question = "iets over geld en beleggen"

In [24]:
docsretriever.invoke(question)

[Document(metadata={'mediacontent_page_description_program': '', 'mediacontent_page_description': 'Wat is een blockchain? Hét woord van de voorbije jaren: blockchain! En na deze video met wiskundige An Braeken (VUB) snap jij ook perfect wat de blockchain is.', 'mediacontent_page_editorialtitle_program': 'Krijg simpele antwoorden op wetenschappelijke vragen', 'mediacontent_pagetitle_program': 'WetenSNAP', 'mediacontent_pagetitle_season': '2022-2023', 'mediacontent_pagetitle': 'Wat is een blockchain?', 'offering_publication_planneduntil': '2032-12-31 22:59:00.000', 'brand_contentbrand': 'radio1', 'mediacontent_pageurl': 'https://www.vrt.be/vrtnu/a-z/wetensnap/2022-2023/wetensnap-s2022-2023a19/', 'mediacontent_imageurl': '//images.vrt.be/orig/2022/10/26/84c3f8c6-5525-11ed-b07d-02b7b76bf47f.png', 'mediacontent_programimageurl': '//images.vrt.be/orig/2022/09/29/bae33113-400c-11ed-b07d-02b7b76bf47f.jpg'}, page_content="Iets met je bankkaart kopen, geld overschrijven of geld storten op je eig

In [21]:
docs = docsearch.similarity_search(
    "geld en beleggen",
    k=5,
    vector_field="vrtmax_catalog_vector"
)

In [22]:
docs

[Document(metadata={'mediacontent_page_description_program': '', 'mediacontent_page_description': 'Duidingsprogramma over de economische actualiteit van de voorbije week.', 'mediacontent_page_editorialtitle_program': 'Duiding bij de economische actualiteit', 'mediacontent_pagetitle_program': 'De markt', 'mediacontent_pagetitle_season': 'Seizoen 2023', 'mediacontent_pagetitle': 'De markt 20230915', 'offering_publication_planneduntil': '2024-09-15 21:55:00.000', 'brand_contentbrand': 'vrtnws', 'mediacontent_pageurl': 'https://www.vrt.be/vrtmax/a-z/de-markt/2023/de-markt-d20230915/', 'mediacontent_imageurl': 'https://images.vrt.be/orig/2023/09/02/297a9680-4913-11ee-91d7-02b7b76bf47f.jpg', 'mediacontent_programimageurl': '//images.vrt.be/orig/2019/10/04/176cfbcb-e6b8-11e9-abcc-02b7b76bf47f.jpg'}, page_content="Obligaties die een rente van 6% beloven. Is dat, na jaren van lage rente, de manier om spaargeld te activeren? Waarom zijn er meer vrouwen in een mannenbastion als de ITsector nodig?

In [6]:
# Generate prompt
system_prompt = """
Je bent een hulpvaardige assistent die Nederlands spreekt.
Je krijgt vragen over de catalogus van programma's van het videoplatform VRT MAX. 
Je probeert mensen te helpen om programma's aan te bevelen die aansluiten bij hun vraag.
Daarvoor krijg je een beschrijving van een aantal programma's.
Aan jou om wat relevant is aan te bevelen.
Begin niet met het geven van jouw mening over de programma's. Begin direct met het aanbevelen van relevante content aan de gebruiker.
"""

user_prompt = ""
for item in out.keys():
    
    user_prompt += "Het programma met de naam " + out[item]["metadata"]["mediacontent_pagetitle_program"] +\
          " heeft volgende beschrijving: " + out[item]["metadata"]["mediacontent_page_description"] 
    
    # + " en in het programma wordt er onder meer het volgende gezegd dat misschien relevant is voor de vraag: " + \out[item]["page_content"]

user_prompt += " Tot zover alle informatie over de programmas."

user_prompt += " Gelieve met die informatie een antwoord te geven op volgende vraag: " + question

In [7]:
# Ask question
# Prompt to generate
messages=[
    ( "system", system_prompt ),
    ( "human", user_prompt )
  ]

stream = llm.stream(messages)
full = next(stream)
for chunk in stream:
    full += chunk
full

AIMessageChunk(content=[{'type': 'text', 'text': 'Als je geïnteresseerd bent in programma\'s over geld en beleggen, zou ik je aanbevelen om eens te kijken naar "Universiteit van Vlaanderen" en "Schermtijd". Beide programma\'s behandelen thema\'s rondom beleggen en financiën, waarbij "Universiteit van Vlaanderen" specifiek ingaat op de wereld van bitcoins en virtueel geld, en "Schermtijd" zich richt op veilig beleggen via beleggingsapps.', 'index': 0}], response_metadata={'stopReason': 'end_turn', 'metrics': {'latencyMs': 2199}}, id='run-fff28f28-e1e0-4010-990f-f52bd11f7d5d', usage_metadata={'input_tokens': 572, 'output_tokens': 115, 'total_tokens': 687})

In [12]:
chunk

AIMessageChunk(content=[], response_metadata={'metrics': {'latencyMs': 3010}}, id='run-bbbf8a1d-5a5c-452f-8e5d-bf91b7d08522', usage_metadata={'input_tokens': 572, 'output_tokens': 167, 'total_tokens': 739})

In [13]:
stream

<generator object BaseChatModel.stream at 0x7259ea1d97e0>